In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras

In [51]:
from keras.layers import Dense,LSTM,Embedding
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import RepeatVector,TimeDistributed

In [4]:
d=pd.read_csv('https://assets.datacamp.com/production/repositories/4605/datasets/73a508c78b5990ab680accf7fd13adcc879b975a/en-pt-sample5k.txt',header=None,sep='\t',names=['English','Frense'])

In [5]:
d.head()

,English,Frense
0,its my job,é o meu trabalho
1,wholl cook,quem cozinhará
2,help me,ajudemme
3,i sat down,eu me sentei
4,im worn out,estou exausto


In [14]:
x=list(d['English'])
y=list(d['Frense'])

In [15]:
eng_token=Tokenizer()
french_tokens=Tokenizer()

In [16]:
eng_token.fit_on_texts(x)
french_tokens.fit_on_texts(y)

In [17]:
x=eng_token.texts_to_sequences(x)
y=french_tokens.texts_to_sequences(y)

In [13]:
length=max(len(sent.split()) for sent in x)

In [18]:
x=pad_sequences(x,maxlen=length,padding='post')
y=pad_sequences(y,maxlen=length,padding='post')

In [39]:
french_vocab_len=len(french_tokens.index_word)
eng_vocab_len=len(eng_token.index_word)

In [21]:
ylist=[]
for seq in y:
  encoded=to_categorical(seq,num_classes=french_vocab_len+1)
  ylist.append(encoded)

In [24]:
y=np.array(ylist)

In [23]:
import numpy as np

In [25]:
y.shape

(5000, 5, 2280)

In [35]:
con=open('/content/glove.6B.50d.txt',encoding='utf-8')

In [36]:
index={}
for line in con:
  values=line.split()
  word=values[0]
  vect=np.asarray(values[1:],dtype='float32')
  index[word]=vect

In [48]:
matrix=np.zeros((eng_vocab_len+1,50))

In [49]:
for word,i in eng_token.word_index.items():
  if i <eng_vocab_len:
    vect=index.get(word)
    if vect is not None:
      matrix[i]=vect

In [53]:
model=Sequential()
model.add(Embedding(eng_vocab_len+1,50,input_length=length,weights=[matrix]))
model.add(LSTM(128))
model.add(RepeatVector(length))

model.add(LSTM(128,return_sequences=True))
model.add(TimeDistributed(Dense(french_vocab_len+1,activation='softmax')))
model.compile(loss='categorical_crossentropy',optimizer='adam')


In [54]:
x.shape
y.shape

(5000, 5, 2280)

In [55]:
x.shape

(5000, 5)

In [56]:
model.fit(x,y,epochs=300)

Epoch 1/300
157/157 [==============================] - 35s 7ms/step - loss: 5.0617
Epoch 2/300
157/157 [==============================] - 1s 7ms/step - loss: 3.4646
Epoch 3/300
157/157 [==============================] - 1s 6ms/step - loss: 3.3524
Epoch 4/300
157/157 [==============================] - 1s 7ms/step - loss: 3.2563
Epoch 5/300
157/157 [==============================] - 1s 6ms/step - loss: 3.1580
Epoch 6/300
157/157 [==============================] - 1s 7ms/step - loss: 3.0548
Epoch 7/300
157/157 [==============================] - 1s 7ms/step - loss: 2.9883
Epoch 8/300
157/157 [==============================] - 1s 7ms/step - loss: 2.8815
Epoch 9/300
157/157 [==============================] - 1s 7ms/step - loss: 2.7936
Epoch 10/300
157/157 [==============================] - 1s 7ms/step - loss: 2.6776
Epoch 11/300
157/157 [==============================] - 1s 7ms/step - loss: 2.5798
Epoch 12/300
157/157 [==============================] - 1s 7ms/step - loss: 2.4881
Epoch 13/300

In [57]:
a=x[0]

In [58]:
a

array([ 11,  60, 269,   0,   0], dtype=int32)

In [63]:
a=d['English'][0]

In [65]:
a=eng_token.texts_to_sequences([a])
a=pad_sequences(a,maxlen=length,padding='post')

In [66]:
a

array([[ 11,  60, 269,   0,   0]], dtype=int32)

In [68]:
b=model.predict(a)

In [69]:
b.shape

(1, 5, 2280)

In [71]:
b[0].shape

(5, 2280)

In [72]:
b=b[0]

In [73]:
b.shape

(5, 2280)

In [75]:
indices=[np.argmax(vect) for vect in b]

In [77]:
text=[]
for i in indices:
  word=french_tokens.index_word.get(i,None)
  if word is None:
    break
  
  text.append(word)

In [78]:
' '.join(text)

'é o meu trabalho'